In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.6.1


In [2]:
!python -V

Python 3.7.0


In [3]:
import pickle
import pandas as pd

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

Run this notebook for the March 2023 data.

In [10]:
year = 2023
month = 3

In [ ]:
df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet')

In [8]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

# Q1. Notebook

What's the standard deviation of the predicted duration for this dataset?

In [9]:
y_pred.std()

6.247488852238703

**Ans:** 6.25

# Q2. Preparing the output

First, let's create an artificial `ride_id` column:

In [12]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

Next, write the ride id and the predictions to a dataframe with results.

In [13]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred

Save it as parquet:

In [14]:
!mkdir output_file

In [15]:
output_file = f'output_file/result_yellow_tripdata_{year:04d}-{month:02d}.parquet'

In [16]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

What's the size of the output file?

In [18]:
!ls -lh output_file

total 134072
-rw-r--r--  1 nop  staff    65M Jun 13 00:05 result_yellow_tripdata_2023-03.parquet


**Ans:** 65M

# Q3. Creating the scoring script

Now let's turn the notebook into a script.

Which command you need to execute for that?

In [ ]:
!jupyter nbconvert --to script homework.ipynb

# Q4. Virtual environment

Now let's put everything into a virtual environment. We'll use pipenv for that.

Install all the required libraries. Pay attention to the Scikit-Learn version: it should be the same as in the starter notebook.

In [ ]:
!pip install --user pipenv

In [ ]:
!pipenv install -r requirements.txt

The `Pipfile.lock` file keeps the hashes of the dependencies we use for the virtual env.

What's the first hash for the Scikit-Learn dependency?

In [27]:
import json

with open("Pipfile.lock", "r") as f:
    pipfile = json.load(f)

pipfile["default"]["scikit-learn"]["hashes"][0]

'sha256:057b991ac64b3e75c9c04b5f9395eaf19a6179244c089afdebaad98264bff37c'

**Ans:** sha256:057b991ac64b3e75c9c04b5f9395eaf19a6179244c089afdebaad98264bff37c

# Q5. Parametrize the script

Let's now make the script configurable via CLI. We'll create two parameters: year and month.

Run the script for April 2023.

What's the mean predicted duration?

In [ ]:
!pipenv run python solution.py 2023 4

**Ans:** 14.29

# Q6. Docker container

Now run the script with docker.

In [ ]:
!docker build -t yellow-taxi-trip-record:v1 -f homework.dockerfile .

In [ ]:
!docker run --rm yellow-taxi-trip-record:v1 2023 5

**Ans:** 14.24